<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

from openai import OpenAI

import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.21.0

Compiler    : Clang 15.0.0 (clang-1500.1.0.2.5)
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 529fe8a3ea9769b549bab95c993473f8266299a8

pandas    : 2.1.4
watermark : 2.4.3
tqdm      : 4.66.2
json      : 2.0.9
matplotlib: 3.8.0
termcolor : 2.4.0
numpy     : 1.26.4



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

Then we are ready to instantiate the model

In [4]:
client = OpenAI()

We start by getting a list of supported models.

In [12]:
model_list = json.loads(client.models.list().json())["data"]

In total we have 32 models

In [13]:
len(model_list)

32

Along with some information about each model...

In [15]:
model_list[:3]

[{'id': 'dall-e-3',
  'created': 1698785189,
  'object': 'model',
  'owned_by': 'system'},
 {'id': 'whisper-1',
  'created': 1677532384,
  'object': 'model',
  'owned_by': 'openai-internal'},
 {'id': 'davinci-002',
  'created': 1692634301,
  'object': 'model',
  'owned_by': 'system'}]

But let's just get a list of model names

In [17]:
print("\n".join(sorted([model.id for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-1106-vision-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4-vision-preview
gpt-4o
gpt-4o-2024-05-13
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [22]:
%%time
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

CPU times: user 5.94 ms, sys: 1.97 ms, total: 7.9 ms
Wall time: 1.19 s


Which produces a response object

In [23]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a named tuple

In [25]:
pprint(response.choices)

ChatCompletion(id='chatcmpl-9QxJmIEDDt1YUzxKcxCR0NRqTDsbt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's weakness is Kryptonite, a mineral from his home planet Krypton that emits radiation that is harmful to him and can greatly weaken and even kill him.", role='assistant', function_call=None, tool_calls=None))], created=1716211574, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=34, prompt_tokens=13, total_tokens=47))


The model answer can be found in the "message" dictionary inside the "choices" list

In [27]:
response.choices[0].message.content

"Superman's weakness is Kryptonite, a mineral from his home planet Krypton that emits radiation that is harmful to him and can greatly weaken and even kill him."

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [28]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 6.93 ms, sys: 1.68 ms, total: 8.6 ms
Wall time: 7.04 s


And we can access each of the answers individually int he choices list

In [29]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
There are several different kinds of Kryptonite in the DC Comics universe, each with its own specific effects on Kryptonians such as Superman. Some of the most common types include:

1. Green Kryptonite: This is the most well-known and common form of Kryptonite, and it is deadly to Kryptonians. Exposure to green Kryptonite weakens them, drains their powers, and can eventually kill them if they are exposed for too long without intervention.

2. Red Kryptonite: This type of Kryptonite causes unpredictable and temporary changes in Kryptonians. It can give them temporary new powers, alter their personalities, or cause other unpredictable effects.

3. Blue Kryptonite: Blue Kryptonite has a calming effect on Kryptonians and can neutralize the harmful effects of green Kryptonite. It is often used as a defense against Superman or other Kryptonian enemies.

4. Gold Kryptonite: This type of Kryptonite permanently removes a Kryptonian's powers, leaving them permanently powerless. It is 

In [31]:
response.usage

CompletionUsage(completion_tokens=778, prompt_tokens=17, total_tokens=795)

# Temperature

In [36]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=1.5
)

CPU times: user 6.63 ms, sys: 1.99 ms, total: 8.61 ms
Wall time: 4.39 s


In [37]:
print(response.choices[0].message.content)

Once there was a little girl named Emma who loved to paint. She would spend hours every day creating beautiful masterpieces on small canvases she had in her tiny room. One day, Emma decided to enter a local art contest that would showcase the work of young artists in her town.

She spent weeks working on a painting that she poured all her heart and soul into. Finally, the day of the contest arrived, and Emma nervously presented her painting among the other entries. The judges were impressed by her intricate brushwork and the depth of emotion in her painting.

When the winners were announced, Emma's name was called to receive the first prize. She was overjoyed and couldn't believe that her hard work had paid off. From that day on, Emma's painting gained recognition in her town, and she eventually became a well-known artist worldwide.

Emma never forgot the joy she felt that day and continued to paint with the same passion and dedication that had propelled her to success.RelativeTo=Emma


In [38]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 5.65 ms, sys: 2.57 ms, total: 8.22 ms
Wall time: 3.86 s


In [39]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lily. Lily was known for her kindness and generosity, always willing to help those in need.

One day, a terrible storm hit the village, causing widespread damage and leaving many families homeless. Lily knew she had to do something to help. She gathered her friends and together they started a relief effort, collecting food, clothing, and supplies for those affected by the storm.

As word spread of Lily's efforts, more and more people in the village joined in, donating whatever they could to help their neighbors in need. Soon, the village was buzzing with activity as everyone worked together to rebuild and support those who had lost everything.

Thanks to Lily's leadership and the community's generosity, the village was able to recover from the storm stronger than ever before. And from that day on, Lily was known as a hero, a shining example of how one person's kindness and compassion can make 

# Function Calls

In [40]:
def chat(messages, functions):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        # Define the functions the model is allowed to use
        functions=functions
    )
    
    return response

In [55]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        print(message)
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "assistant" and message['function_call']:
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message['role']]))
        elif message["role"] == "assistant" and not message['function_call']:
            print(colored(f"assistant: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "function":
            print(colored(f"function ({message.name}): {message.content}\n", role_to_color[message.role]))


Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [51]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [59]:
messages = []

messages.append(
    {"role": "system", 
     "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    })

messages.append(
    {"role": "user", 
     "content": "What's the weather like today"
    })

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [24]:
messages.append(
    {"role": "user", 
     "content": "I'm in New York, NY."
    })

In [25]:
chat_response = chat(messages, functions=functions)
assistant_message = chat_response.choices[0].message
messages.append({
 "role":  assistant_message.role,
 "content":  assistant_message.content,
 "function_call":  assistant_message.function_call,
})

BadRequestError: Error code: 400 - {'error': {'message': "None is not of type 'object' - 'messages.2.function_call'", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [26]:
pretty_print_conversation(messages)

{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

{'role': 'user', 'content': "What's the weather like today"}
user: What's the weather like today

{'role': 'user', 'content': "I'm in New York, NY."}
user: I'm in New York, NY.

{'role': 'assistant', 'content': None, 'function_call': FunctionCall(arguments='{"location":"New York, NY","format":"celsius"}', name='get_current_weather')}
assistant: FunctionCall(arguments='{"location":"New York, NY","format":"celsius"}', name='get_current_weather')



## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [27]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

This sudden change in direction means we don't have time to do everything for the client.


# Formatted output

In [28]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = client.chat.completions.create(
          model = "gpt-3.5-turbo",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 7.97 ms, sys: 2.11 ms, total: 10.1 ms
Wall time: 4.84 s


In [29]:
json_output = response.choices[0].message.content

In [30]:
output = json.loads(json_output)

In [31]:
output["ingredients"]

['1 cup all-purpose flour',
 '1 tablespoon sugar',
 '1 teaspoon baking powder',
 '1/2 teaspoon baking soda',
 '1/2 teaspoon salt',
 '1 cup buttermilk',
 '1 large egg',
 '2 tablespoons unsalted butter, melted',
 '1 cup fresh blueberries',
 'Butter or oil for cooking']

In [32]:
output["instructions"]

['In a large bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.',
 'In a separate bowl, whisk together the buttermilk, egg, and melted butter.',
 'Add the wet ingredients to the dry ingredients and stir until just combined. Be careful not to overmix, as lumps are fine.',
 'Gently fold in the blueberries.',
 'Heat a skillet or griddle over medium heat and add a small amount of butter or oil.',
 'Pour 1/4 cup of batter onto the skillet for each pancake.',
 'Cook until bubbles form on the surface of the pancake, then flip and cook for another minute or until golden brown.',
 'Repeat with the remaining batter, adding more butter or oil to the skillet as needed.',
 'Serve the blueberry pancakes warm with maple syrup or your favorite pancake toppings.']

# Translation

In [33]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [34]:
response.choices[0].message.content

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [35]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [36]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [37]:
print(response.choices[0].message.content)

Fruit,Color,Flavor
neoskizzles,Purple,Candy
loheckles,Grayish blue,Tart
pounits,Bright green,Savory
loopnovas,Neon pink,Cotton candy
glowls,Pale orange,Sour and bitter


In [38]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [39]:
print(response.choices[0].message.content)

Pounits are bright green fruits that are more savory than sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>